In [2]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
tf.random.set_seed(10)
import os.path
import pandas as pd
import time
import requests
import json
import numpy as np
import talib
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)

In [3]:
def get_klinedata(platform, symbol, granularity):
    if platform == "huobi":  # 2000条数据，时间粒度1min, 5min, 15min, 30min, 60min, 4hour, 1day, 1mon, 1week, 1year
        huobi_granularity_dict = {60: "1min", 300: "5min", 900: "15min", 1800: "30min", 3600: "60min",
                                  14400: "4hour", 86400: "1day", 604800: "1week", 2592000: "mon",
                                  946080000: "1year"}
        for _ in range(3):
            try:
                res = requests.get("https://api.huobi.pro/market/history/kline?period={}&size=2000&symbol={}".format(
                    huobi_granularity_dict[granularity], symbol.replace("_", "")), timeout=1)
                data = json.loads(res.content.decode())["data"]
                df=pd.DataFrame()
                if res.status_code==200:
                    data = json.loads(res.content.decode())["data"][::-1]
                    df['close'] = [i['close'] for i in data]
                    df['high'] =[i['high'] for i in data]
                    df['low'] =[i['low'] for i in data]
                    df['open'] = [i['open'] for i in data]
                    df['volume']= [i['vol'] for i in data]
                    df['time'] = [time.strftime("%Y-%m-%d", time.localtime(i['id'])) for i in data]
                    break
            except Exception as e:
                print(e)
                df = pd.DataFrame()
        return df

In [219]:
#数据去极值及标准化
def winsorize_and_standarlize(data,qrange=[0.05,0.95],axis=0):
    '''
    input:
    data:Dataframe or series,输入数据
    qrange:list,list[0]下分位数，list[1]，上分位数，极值用分位数代替
    '''
    if isinstance(data,pd.DataFrame):
        if axis == 0:
            q_down = data.quantile(qrange[0])
            q_up = data.quantile(qrange[1])
            index = data.index
            col = data.columns
            for n in col:
                data[n][data[n] > q_up[n]] = q_up[n]
                data[n][data[n] < q_down[n]] = q_down[n]
            data = (data - data.mean())/data.std()
            data = data.fillna(0)
        else:
            data = data.stack()
            data = data.unstack(0)
            q = data.quantile(qrange)
            index = data.index
            col = data.columns
            for n in col:
                data[n][data[n] > q[n]] = q[n]
            data = (data - data.mean())/data.std()
            data = data.stack().unstack(0)
            data = data.fillna(0)
            
    elif isinstance(data,pd.Series):
        name = data.name
        q = data.quantile(qrange)
        data[data>q] = q
        data = (data - data.mean())/data.std()
    return data

In [228]:
def get_factor(symbol):
    df = get_klinedata("huobi", symbol, granularity=86400)
    df['symbol'] = symbol
    df['MOM_10'] = df['close'] / df['close'].shift(9) - 1
    df['MOM_60'] = df['close'] / df['close'].shift(59) - 1
    df['MOM_120'] = df['close'] / df['close'].shift(119) - 1
    df['VSTD_20'] = df['volume'].rolling(20).std()
    df['std_20'] = df['close'].rolling(20).std()
    df['std_40'] = df['close'].rolling(40).std()
    df['std_120'] = df['close'].rolling(120).std()
    df['MA_5'] = df['close'].rolling(5).mean()
    df['MA_20'] = df['close'].rolling(10).mean()
    df['MA_Cross'] = np.where(df['MA_5'] > df['MA_20'], 1, 0)
    df['label'] = np.where(df['close'] > df['close'].shift(-7), 0, 1)
    # 复合动量因子
    df['ret'] = np.log(df['close'] / df['close'].shift())
    df['ret_60'] = df['ret'].rolling(60).sum()
    # 收益率偏度因子
    df['skew_60'] = df['ret'].rolling(60).skew()
    df['MACD'] = talib.MACD(df.close, fastperiod=6, slowperiod=12, signalperiod=9)[-1]
    df['RSI'] = talib.RSI(df.close, timeperiod=12)
    df['MOM'] = talib.MOM(df.close, timeperiod=5)
    df['CCI'] = talib.CCI(df.high, df.low, df.close, timeperiod=14)
    df['WILLR'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=14)
    df['SAR'] = talib.SAR(df.high, df.low)
    df['OBV']=talib.OBV(df.close,df.volume)
    df['ADOSC'] = talib.ADOSC(df.high, df.low, df.close,df.volume)
    df['ROC']=talib.ROC(df.volume) #新增1
    df['SLOWK'],df['SLOWD']=talib.STOCH(df.high, df.low, df.close)
    df['HT']=talib.HT_TRENDLINE(df.close)
    df['ADX']=talib.ADX(df.high,df.low,df.close)
    df['APO']=talib.APO(df.close)
    df['AROONDOWN'],df['AROONUP']=talib.AROON(df.high,df.low)
    df['AROONOSC']=talib.AROONOSC(df.high,df.low)
    df['BOP']=talib.BOP(df.open,df.high,df.low,df.close)
    df['CMO']=talib.CMO(df.close)
    df['MFI']=talib.MFI(df.high,df.low,df.close,df.volume)
    df['PPO']=talib.PPO(df.close)
    df['ULTOSC']=talib.ULTOSC(df.high,df.low,df.close) #新增2
    df['AD']=talib.AD(df.high,df.low,df.close,df.volume)
    df['COL3BLACKCROWS']=talib.CDL3BLACKCROWS(df.open, df.high, df.low, df.close)
    df['CDLDOJI']=talib.CDLDOJI(df.open,df.high,df.low,df.close)
    df['CDLENGULFING']=talib.CDLENGULFING(df.open, df.high, df.low, df.close)
    df['CDLHAMMER']=talib.CDLHAMMER(df.open, df.high, df.low, df.close)
    df['CDLMORNINGSTAR']=talib.CDLMORNINGSTAR(df.open, df.high, df.low, df.close)
    df['TSF']=talib.TSF(df.close)
    df.dropna(inplace=True)
    return df

In [229]:
def get_symbols_factor(symbols):
    data= pd.DataFrame()
    for symbol in symbols:
        symbol = "{}_usdt".format(symbol)
        df = get_factor(symbol)
        data= pd.concat([data, df], axis=0)
    return data

In [230]:
huobifuture_api_url='https://api.btcgateway.pro'
symbols = ['btc', 'eth', 'link', 'eos', 'fil', 'ltc', 'dot', 'doge','bsv','ada','etc','trx','xmr']
df_sum=get_symbols_factor(symbols)

In [231]:
df_sum=df_sum.sample(frac=1.0,random_state=1)#打乱所有数据
df_sum=df_sum.reset_index(drop=True)

In [260]:
split=int(len(df_sum)*0.7)
train=df_sum.iloc[:split]
test=df_sum.iloc[split:]
features =df_sum.columns.difference(['close','high','low','open','volume','time','label','ret','MA_5','MA_20','symbol',
                                     ])

In [261]:
pca=PCA(n_components=35)
x_train=np.array(winsorize_and_standarlize(train[features]))
y_train=np.array(train.label)
x_test=np.array(winsorize_and_standarlize(test[features]))
y_test=np.array(test.label)

# Keras 训练

In [226]:
x_train = tf.keras.utils.normalize(x_train, axis=1).reshape(x_train.shape[0], -1)
x_test = tf.keras.utils.normalize(x_test, axis=1).reshape(x_test.shape[0], -1)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())   #Flatten the images! Could be done with numpy reshape
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu, input_shape= x_train.shape[1:]))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.sigmoid))   #2 because dataset is numbers from 0 - 1
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),  # Good default optimizer to start with
              loss='sparse_categorical_crossentropy',  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'],
              )  # what to track

In [227]:
import time
start_time=time.time()
#model.load_weights('store_data/dnn_weights')
model.fit(x_train, y_train, validation_data=(x_test,y_test),epochs=200,batch_size=64,shuffle=True)  # train the model
model.save_weights('store_data/dnn_weights')
end_time=time.time()
val_loss, val_acc =model.evaluate(x_test, y_test)
print(f"共耗时{end_time-start_time}秒")

Epoch 1/200
178/178 [==============================] - 1s 4ms/step - loss: 0.6917 - accuracy: 0.5284 - val_loss: 0.6919 - val_accuracy: 0.5213
Epoch 2/200
178/178 [==============================] - 0s 3ms/step - loss: 0.6906 - accuracy: 0.5327 - val_loss: 0.6912 - val_accuracy: 0.5213
Epoch 3/200
178/178 [==============================] - 0s 3ms/step - loss: 0.6892 - accuracy: 0.5365 - val_loss: 0.6908 - val_accuracy: 0.5284
Epoch 4/200
178/178 [==============================] - 0s 3ms/step - loss: 0.6879 - accuracy: 0.5442 - val_loss: 0.6903 - val_accuracy: 0.5355
Epoch 5/200
178/178 [==============================] - 0s 3ms/step - loss: 0.6863 - accuracy: 0.5511 - val_loss: 0.6893 - val_accuracy: 0.5458
Epoch 6/200
178/178 [==============================] - 0s 3ms/step - loss: 0.6824 - accuracy: 0.5616 - val_loss: 0.6878 - val_accuracy: 0.5624
Epoch 7/200
178/178 [==============================] - 0s 3ms/step - loss: 0.6817 - accuracy: 0.5626 - val_loss: 0.6868 - val_accuracy: 0.5600

Epoch 58/200
178/178 [==============================] - 0s 3ms/step - loss: 0.5802 - accuracy: 0.6890 - val_loss: 0.6036 - val_accuracy: 0.6603
Epoch 59/200
178/178 [==============================] - 1s 3ms/step - loss: 0.5789 - accuracy: 0.6898 - val_loss: 0.6029 - val_accuracy: 0.6698
Epoch 60/200
178/178 [==============================] - 1s 3ms/step - loss: 0.5720 - accuracy: 0.6969 - val_loss: 0.6014 - val_accuracy: 0.6682
Epoch 61/200
178/178 [==============================] - 0s 3ms/step - loss: 0.5743 - accuracy: 0.6972 - val_loss: 0.6040 - val_accuracy: 0.6619
Epoch 62/200
178/178 [==============================] - 1s 4ms/step - loss: 0.5708 - accuracy: 0.6992 - val_loss: 0.6021 - val_accuracy: 0.6690
Epoch 63/200
178/178 [==============================] - 1s 4ms/step - loss: 0.5694 - accuracy: 0.7013 - val_loss: 0.6017 - val_accuracy: 0.6667
Epoch 64/200
178/178 [==============================] - 0s 3ms/step - loss: 0.5690 - accuracy: 0.7058 - val_loss: 0.6012 - val_accuracy:

Epoch 115/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4846 - accuracy: 0.7590 - val_loss: 0.5731 - val_accuracy: 0.6998
Epoch 116/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4816 - accuracy: 0.7662 - val_loss: 0.5795 - val_accuracy: 0.6912
Epoch 117/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4816 - accuracy: 0.7665 - val_loss: 0.5761 - val_accuracy: 0.6927
Epoch 118/200
178/178 [==============================] - 1s 3ms/step - loss: 0.4823 - accuracy: 0.7659 - val_loss: 0.5761 - val_accuracy: 0.6983
Epoch 119/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4821 - accuracy: 0.7630 - val_loss: 0.5792 - val_accuracy: 0.6927
Epoch 120/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4786 - accuracy: 0.7643 - val_loss: 0.5768 - val_accuracy: 0.6991
Epoch 121/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4774 - accuracy: 0.7705 - val_loss: 0.5759 - val_ac

178/178 [==============================] - 0s 3ms/step - loss: 0.4170 - accuracy: 0.8079 - val_loss: 0.5679 - val_accuracy: 0.7077
Epoch 172/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4110 - accuracy: 0.8110 - val_loss: 0.5702 - val_accuracy: 0.7164
Epoch 173/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4109 - accuracy: 0.8106 - val_loss: 0.5798 - val_accuracy: 0.7133
Epoch 174/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4056 - accuracy: 0.8095 - val_loss: 0.5838 - val_accuracy: 0.7077
Epoch 175/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4169 - accuracy: 0.8062 - val_loss: 0.5699 - val_accuracy: 0.7141
Epoch 176/200
178/178 [==============================] - 0s 3ms/step - loss: 0.4020 - accuracy: 0.8152 - val_loss: 0.5731 - val_accuracy: 0.7148
Epoch 177/200
178/178 [==============================] - 1s 3ms/step - loss: 0.4066 - accuracy: 0.8133 - val_loss: 0.5734 - val_accuracy: 0.7156

In [ ]:
model.save_weights('store_data/dnn_weights')

# SVM预测

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
model_svm=SVC(C=10000)

In [ ]:
model = Pipeline([
    #('pca', PCA()),
    ('svc', model_svm)])
model.fit(winsorize_and_standarlize(train[features]),train.label)

In [ ]:
a=accuracy_score(train.label,model.predict(winsorize_and_standarlize(train[features])))
b=accuracy_score(test.label,model.predict(winsorize_and_standarlize(test[features])))
print(a,b)

In [ ]:
features

In [262]:
main_df =winsorize_and_standarlize(train[['AD', 'ADOSC', 'ADX', 'APO', 'AROONDOWN', 'AROONOSC', 'AROONUP', 'BOP',
       'CCI', 'CDLDOJI', 'CDLENGULFING', 'CDLHAMMER', 'CDLMORNINGSTAR', 'CMO',
       'COL3BLACKCROWS', 'HT', 'MACD', 'MA_Cross', 'MFI', 'MOM', 'MOM_10',
       'MOM_120', 'MOM_60', 'OBV', 'PPO', 'ROC', 'RSI', 'SAR', 'SLOWD',
       'SLOWK', 'TSF', 'ULTOSC', 'VSTD_20', 'WILLR', 'ret_60', 'skew_60',
       'std_120', 'std_20', 'std_40']])  # now the main_df is all the data up to the last 5%

In [263]:
main_df=main_df.join(train['label'])

In [264]:
validation_main_df=winsorize_and_standarlize(test[['AD', 'ADOSC', 'ADX', 'APO', 'AROONDOWN', 'AROONOSC', 'AROONUP', 'BOP',
       'CCI', 'CDLDOJI', 'CDLENGULFING', 'CDLHAMMER', 'CDLMORNINGSTAR', 'CMO',
       'COL3BLACKCROWS', 'HT', 'MACD', 'MA_Cross', 'MFI', 'MOM', 'MOM_10',
       'MOM_120', 'MOM_60', 'OBV', 'PPO', 'ROC', 'RSI', 'SAR', 'SLOWD',
       'SLOWK', 'TSF', 'ULTOSC', 'VSTD_20', 'WILLR', 'ret_60', 'skew_60',
       'std_120', 'std_20', 'std_40']])

In [265]:
validation_main_df=validation_main_df.join(test['label'])

# RNN训练

In [266]:
def preprocess_df(df):
    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in df.values:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target
        if len(prev_days) == SEQ_LEN:  # make sure we have 60 sequences!
            sequential_data.append([np.array(prev_days), i[-1]])  # append those bad boys!

    random.shuffle(sequential_data)  # shuffle for good measure.
    
    buys=[]
    sells=[]
    
    for seq, target in sequential_data:  # iterate over the sequential data
        if target == 0:  # if it's a "not buy"
            sells.append([seq, target])  # append to sells list
        elif target == 1:  # otherwise if the target is a 1...
            buys.append([seq, target])  # it's a buy!
    
    random.shuffle(buys)  # shuffle the buys
    random.shuffle(sells)  # shuffle the sells!

#     lower = min(len(buys), len(sells))  # what's the shorter length?

#     buys = buys[:lower]  # make sure both lists are only up to the shortest length.
#     sells = sells[:lower]  # make sure both lists are only up to the shortest length.

    sequential_data = buys+sells  # add them together
    random.shuffle(sequential_data)  # another shuffle, so the model doesn't get confused with all 1 class then the other.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), np.array(y)  # return X and y...and make X a numpy array!

In [267]:
from collections import deque
import random
import numpy as np
SEQ_LEN = 64
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [241]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization,RNN
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers

In [246]:
model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
#model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(64))
model.add(Dropout(0.2))
#model.add(BatchNormalization())

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

opt = tf.keras.optimizers.Adam(lr=0.1)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [247]:
model.fit(train_x,train_y,validation_data=(validation_x, validation_y),epochs=200)  # train the model

Epoch 1/200
236/236 [==============================] - 17s 72ms/step - loss: 0.7066 - accuracy: 0.5052 - val_loss: 0.6971 - val_accuracy: 0.4728
Epoch 2/200
236/236 [==============================] - 16s 70ms/step - loss: 0.6961 - accuracy: 0.5169 - val_loss: 0.6917 - val_accuracy: 0.5272
Epoch 3/200
236/236 [==============================] - 17s 71ms/step - loss: 0.6938 - accuracy: 0.5180 - val_loss: 0.6919 - val_accuracy: 0.5272
Epoch 4/200
236/236 [==============================] - 17s 70ms/step - loss: 0.6941 - accuracy: 0.5246 - val_loss: 0.7002 - val_accuracy: 0.5272
Epoch 5/200
236/236 [==============================] - 17s 71ms/step - loss: 0.6977 - accuracy: 0.5196 - val_loss: 0.6969 - val_accuracy: 0.5272
Epoch 6/200
235/236 [============================>.] - ETA: 0s - loss: 0.6944 - accuracy: 0.5078

KeyboardInterrupt: 

In [ ]:
val_loss, val_acc = model.evaluate(validation_x, validation_y)  # evaluate the out of sample data with model
print(val_loss)  # model's loss (error)
print(val_acc)  # model's accuracy
#model.save('LSTM.model')

# CNN训练

In [269]:
x_train=np.array(winsorize_and_standarlize(train[features]))
y_train=np.array(train.label)
x_test=np.array(winsorize_and_standarlize(test[features]))
y_test=np.array(test.label)

In [270]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(8855, 39) (8855,) (3796, 39) (3796,)


In [271]:
X_train=np.expand_dims(x_train,axis=2)
X_test=np.expand_dims(x_test,axis=2)

In [268]:
def create_cnnmodel():
    model=Sequential()
    #建立一维卷积层
    model.add(Conv1D(filters=32,#建立16个滤镜
                     kernel_size=3,
                     padding='same',
                     activation='relu',
                    ))
    #建立一维池化层1
    model.add(MaxPooling1D(pool_size=3)) 
    model.add(Dropout(0.3))
    model.add(Conv1D(filters=32,#建立16个滤镜
                     kernel_size=3,
                     padding='same',
                     activation='relu',
                    ))
    model.add(MaxPooling1D(pool_size=3)) 
    model.add(Conv1D(filters=32,#建立16个滤镜
                     kernel_size=3,
                     padding='same',
                     activation='relu',
                    ))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.3))
    #建立展平层
    model.add(Flatten())
    #建立隐藏层
    model.add(Dense(32,activation='relu'))
    #随机放弃25%的神经元
    model.add(Dropout(0.2))
    
    #输出层
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(lr=0.001,decay=1e-6),
                  metrics=['accuracy'])
    return model

In [272]:
#model3=create_cnnmodel()
model3.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=400)

Epoch 1/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5884 - accuracy: 0.6827 - val_loss: 0.6470 - val_accuracy: 0.6258
Epoch 2/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5912 - accuracy: 0.6745 - val_loss: 0.6439 - val_accuracy: 0.6236
Epoch 3/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5881 - accuracy: 0.6885 - val_loss: 0.6397 - val_accuracy: 0.6345
Epoch 4/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5897 - accuracy: 0.6831 - val_loss: 0.6393 - val_accuracy: 0.6325
Epoch 5/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5870 - accuracy: 0.6804 - val_loss: 0.6398 - val_accuracy: 0.6291
Epoch 6/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5892 - accuracy: 0.6779 - val_loss: 0.6387 - val_accuracy: 0.6287
Epoch 7/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5938 - accuracy: 0.6759 - val_loss: 0.6394 - val_accuracy: 0.6297

Epoch 58/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5883 - accuracy: 0.6835 - val_loss: 0.6270 - val_accuracy: 0.6406
Epoch 59/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5829 - accuracy: 0.6890 - val_loss: 0.6264 - val_accuracy: 0.6467
Epoch 60/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5844 - accuracy: 0.6873 - val_loss: 0.6253 - val_accuracy: 0.6485
Epoch 61/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5867 - accuracy: 0.6788 - val_loss: 0.6263 - val_accuracy: 0.6428
Epoch 62/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5911 - accuracy: 0.6789 - val_loss: 0.6235 - val_accuracy: 0.6451
Epoch 63/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5882 - accuracy: 0.6813 - val_loss: 0.6264 - val_accuracy: 0.6396
Epoch 64/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5811 - accuracy: 0.6893 - val_loss: 0.6219 - val_accuracy:

Epoch 115/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5820 - accuracy: 0.6814 - val_loss: 0.6203 - val_accuracy: 0.6435
Epoch 116/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5796 - accuracy: 0.6894 - val_loss: 0.6186 - val_accuracy: 0.6522
Epoch 117/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5836 - accuracy: 0.6896 - val_loss: 0.6190 - val_accuracy: 0.6483
Epoch 118/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5823 - accuracy: 0.6882 - val_loss: 0.6184 - val_accuracy: 0.6530
Epoch 119/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5852 - accuracy: 0.6827 - val_loss: 0.6193 - val_accuracy: 0.6572
Epoch 120/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5756 - accuracy: 0.6861 - val_loss: 0.6187 - val_accuracy: 0.6501
Epoch 121/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5831 - accuracy: 0.6830 - val_loss: 0.6188 - val_ac

277/277 [==============================] - 1s 4ms/step - loss: 0.5774 - accuracy: 0.6925 - val_loss: 0.6177 - val_accuracy: 0.6532
Epoch 172/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5769 - accuracy: 0.6908 - val_loss: 0.6165 - val_accuracy: 0.6550
Epoch 173/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5804 - accuracy: 0.6865 - val_loss: 0.6182 - val_accuracy: 0.6550
Epoch 174/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5802 - accuracy: 0.6884 - val_loss: 0.6179 - val_accuracy: 0.6568
Epoch 175/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5815 - accuracy: 0.6831 - val_loss: 0.6194 - val_accuracy: 0.6505
Epoch 176/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5824 - accuracy: 0.6923 - val_loss: 0.6159 - val_accuracy: 0.6536
Epoch 177/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5886 - accuracy: 0.6853 - val_loss: 0.6178 - val_accuracy: 0.6540

277/277 [==============================] - 1s 4ms/step - loss: 0.5804 - accuracy: 0.6916 - val_loss: 0.6158 - val_accuracy: 0.6564
Epoch 228/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5795 - accuracy: 0.6898 - val_loss: 0.6146 - val_accuracy: 0.6566
Epoch 229/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5809 - accuracy: 0.6837 - val_loss: 0.6166 - val_accuracy: 0.6536
Epoch 230/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5835 - accuracy: 0.6872 - val_loss: 0.6138 - val_accuracy: 0.6633
Epoch 231/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5759 - accuracy: 0.6890 - val_loss: 0.6180 - val_accuracy: 0.6564
Epoch 232/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5791 - accuracy: 0.6851 - val_loss: 0.6139 - val_accuracy: 0.6590
Epoch 233/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5814 - accuracy: 0.6870 - val_loss: 0.6166 - val_accuracy: 0.6548

277/277 [==============================] - 1s 4ms/step - loss: 0.5761 - accuracy: 0.6920 - val_loss: 0.6126 - val_accuracy: 0.6599
Epoch 284/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5737 - accuracy: 0.6911 - val_loss: 0.6127 - val_accuracy: 0.6550
Epoch 285/400
277/277 [==============================] - 1s 5ms/step - loss: 0.5729 - accuracy: 0.6884 - val_loss: 0.6116 - val_accuracy: 0.6601
Epoch 286/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5722 - accuracy: 0.6929 - val_loss: 0.6141 - val_accuracy: 0.6615
Epoch 287/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5707 - accuracy: 0.6942 - val_loss: 0.6153 - val_accuracy: 0.6568
Epoch 288/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5807 - accuracy: 0.6857 - val_loss: 0.6134 - val_accuracy: 0.6556
Epoch 289/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5800 - accuracy: 0.6874 - val_loss: 0.6152 - val_accuracy: 0.6629

277/277 [==============================] - 1s 4ms/step - loss: 0.5670 - accuracy: 0.6957 - val_loss: 0.6108 - val_accuracy: 0.6627
Epoch 340/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5791 - accuracy: 0.6901 - val_loss: 0.6138 - val_accuracy: 0.6663
Epoch 341/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5745 - accuracy: 0.6907 - val_loss: 0.6129 - val_accuracy: 0.6568
Epoch 342/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5733 - accuracy: 0.6922 - val_loss: 0.6138 - val_accuracy: 0.6635
Epoch 343/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5759 - accuracy: 0.6928 - val_loss: 0.6143 - val_accuracy: 0.6615
Epoch 344/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5759 - accuracy: 0.6927 - val_loss: 0.6154 - val_accuracy: 0.6607
Epoch 345/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5801 - accuracy: 0.6907 - val_loss: 0.6133 - val_accuracy: 0.6641

277/277 [==============================] - 1s 4ms/step - loss: 0.5784 - accuracy: 0.6910 - val_loss: 0.6090 - val_accuracy: 0.6702
Epoch 396/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5703 - accuracy: 0.6898 - val_loss: 0.6087 - val_accuracy: 0.6643
Epoch 397/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5697 - accuracy: 0.6978 - val_loss: 0.6084 - val_accuracy: 0.6661
Epoch 398/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5740 - accuracy: 0.6861 - val_loss: 0.6089 - val_accuracy: 0.6649
Epoch 399/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5709 - accuracy: 0.6970 - val_loss: 0.6080 - val_accuracy: 0.6663
Epoch 400/400
277/277 [==============================] - 1s 4ms/step - loss: 0.5814 - accuracy: 0.6821 - val_loss: 0.6082 - val_accuracy: 0.6665


In [ ]:
predictions = model3.predict(X_test)

In [ ]:
y_test

# CNN+LSTM

In [273]:
from keras.layers import Input, Dense, LSTM
from keras.models import Model
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam

In [275]:
print(train_x.shape)

(8792, 64, 39)


In [280]:
TIME_STEPS=64
INPUT_DIM=39
lstm_units = 64
batch_size = 256
inputs = Input(shape=(TIME_STEPS, INPUT_DIM))
x = Conv1D(filters = 64, kernel_size = 1, activation = 'relu')(inputs)
x = MaxPooling1D(pool_size = 3)(x)
x = Dropout(0.5)(x)

# x = Conv1D(filters = 32, kernel_size = 1, activation = 'relu')(inputs)
# x = MaxPooling1D(pool_size = 3)(x)
# x = Dropout(0.2)(x)

# x = Conv1D(filters = 32, kernel_size = 1, activation = 'relu')(inputs)
# x = MaxPooling1D(pool_size = 3)(x)
# x = Dropout(0.2)(x)
# print(x.shape)
lstm_out = Bidirectional(LSTM(lstm_units, activation='relu'), name='bilstm')(x)
lstm_out=Flatten()(lstm_out)
output = Dense(10, activation='sigmoid')(lstm_out)
output = Dense(2, activation='sigmoid')(output)
model = Model(inputs=inputs, outputs=output)
opt = tf.keras.optimizers.Adam(lr=0.1, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [281]:
model.fit(train_x,train_y, validation_data=(validation_x, validation_y),epochs=60, batch_size=batch_size)

Epoch 1/60
35/35 [==============================] - 3s 97ms/step - loss: 0.7050 - accuracy: 0.5297 - val_loss: 0.6931 - val_accuracy: 0.5216
Epoch 2/60
35/35 [==============================] - 3s 85ms/step - loss: 0.6930 - accuracy: 0.5349 - val_loss: 0.6930 - val_accuracy: 0.5216
Epoch 3/60
35/35 [==============================] - 3s 84ms/step - loss: 0.6928 - accuracy: 0.5349 - val_loss: 0.6929 - val_accuracy: 0.5216
Epoch 4/60
35/35 [==============================] - 3s 86ms/step - loss: 0.6926 - accuracy: 0.5349 - val_loss: 0.6927 - val_accuracy: 0.5216
Epoch 5/60
35/35 [==============================] - 3s 85ms/step - loss: 0.6922 - accuracy: 0.5349 - val_loss: 0.6924 - val_accuracy: 0.5216
Epoch 6/60
35/35 [==============================] - 3s 86ms/step - loss: 0.6912 - accuracy: 0.5349 - val_loss: 0.6928 - val_accuracy: 0.5216
Epoch 7/60
35/35 [==============================] - 3s 86ms/step - loss: 0.6909 - accuracy: 0.5349 - val_loss: 0.6926 - val_accuracy: 0.5216
Epoch 8/60
35

KeyboardInterrupt: 

# CNN+LSTM+ATTENTION

In [293]:
from keras.layers import Input, Dense, merge
from keras import layers

inputs = Input(shape=(TIME_STEPS, INPUT_DIM))
x = Conv1D(filters = 32, kernel_size = 3, activation = 'relu')(inputs)  #, padding = 'same'
x = MaxPooling1D(pool_size = 3)(x)
x = Dropout(0.2)(x)
print(x.shape)
#lstm_out = Bidirectional(LSTM(lstm_units, activation='relu'), name='bilstm')(x)
lstm_out = LSTM(64,activation='relu',return_sequences=True)(x)
lstm_out = LSTM(32,activation='relu',return_sequences=True)(lstm_out )
lstm_out = LSTM(16,activation='relu')(lstm_out )
print(lstm_out.shape)

(None, 20, 32)
(None, 16)


In [294]:
# ATTENTION PART STARTS HERE
attention_probs = Dense(16, activation='sigmoid', name='attention_vec')(lstm_out)
#attention_mul=layers.merge([stm_out,attention_probs], output_shape],mode='concat',concat_axis=1))
attention_mul =Multiply()([lstm_out, attention_probs])
#attention_mul = merge([lstm_out, attention_probs],output_shape=32, name='attention_mul', mode='mul')

In [295]:
output = Dense(64, activation='sigmoid')(attention_mul)
output =Dropout(0.2)(output)
output = Dense(2, activation='sigmoid')(output)

model = Model(inputs=inputs, outputs=output)
print(model.summary())

Model: "functional_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, 64, 39)]     0                                            
__________________________________________________________________________________________________
conv1d_103 (Conv1D)             (None, 62, 32)       3776        input_27[0][0]                   
__________________________________________________________________________________________________
max_pooling1d_103 (MaxPooling1D (None, 20, 32)       0           conv1d_103[0][0]                 
__________________________________________________________________________________________________
dropout_180 (Dropout)           (None, 20, 32)       0           max_pooling1d_103[0][0]          
______________________________________________________________________________________

In [296]:
opt = tf.keras.optimizers.Adam(lr=1)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [297]:
model.fit(train_x,train_y, validation_data=(validation_x, validation_y),batch_size=64,epochs=200)

Epoch 1/200
138/138 [==============================] - 5s 33ms/step - loss: nan - accuracy: 0.4645 - val_loss: nan - val_accuracy: 0.4784
Epoch 2/200
138/138 [==============================] - 4s 26ms/step - loss: nan - accuracy: 0.4651 - val_loss: nan - val_accuracy: 0.4784
Epoch 3/200
138/138 [==============================] - 4s 27ms/step - loss: nan - accuracy: 0.4651 - val_loss: nan - val_accuracy: 0.4784
Epoch 4/200
138/138 [==============================] - 4s 28ms/step - loss: nan - accuracy: 0.4651 - val_loss: nan - val_accuracy: 0.4784
Epoch 5/200
138/138 [==============================] - 4s 27ms/step - loss: nan - accuracy: 0.4651 - val_loss: nan - val_accuracy: 0.4784
Epoch 6/200
138/138 [==============================] - 4s 27ms/step - loss: nan - accuracy: 0.4651 - val_loss: nan - val_accuracy: 0.4784
Epoch 7/200
138/138 [==============================] - 4s 28ms/step - loss: nan - accuracy: 0.4651 - val_loss: nan - val_accuracy: 0.4784
Epoch 8/200
  6/138 [>............

KeyboardInterrupt: 